In [1]:
import os

import pandas as pd

In [2]:
df = pd.read_csv('../data/processed/user_data.csv')

In [3]:
df.shape

(20621, 11)

In [4]:
products = pd.read_csv('../data/raw/products.csv')
product_to_aisle = dict(zip(products['product_id'], products['aisle_id']))
product_to_department = dict(zip(products['product_id'], products['department_id']))
product_to_name = dict(zip(products['product_id'], products['product_name']))

In [5]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
user_ids = []
product_ids = []
aisle_ids = []
department_ids = []
product_names = []
eval_sets = []

In [7]:
is_ordered_histories = []
index_in_order_histories = []
order_size_histories = []
reorder_size_histories = []
order_dow_histories = []
order_hour_histories = []
days_since_prior_order_histories = []
order_number_histories = []

In [8]:
labels = []

In [10]:
longest = 0

In [16]:
# Extract one row from df.
# Instead of 'for _, row in df.iterrows():
row = next(df.iterrows())[1]
row

user_id                                                                    5
order_ids                             2717275 1909121 2267326 157374 2196797
order_numbers                                                      1 2 3 4 5
order_dows                                                         3 0 3 1 0
order_hours                                                   12 16 18 18 11
days_since_prior_orders                                         0 11 10 19 6
product_ids                15349_21413_48775_28289_8518_11777_31717_26604...
aisle_ids                  29_66_83_123_83_123_16_24_84_83_120 84_12_120_...
department_ids             13_6_4_4_4_4_4_4_16_4_16 16_9_16_4_4_13_4_19_4...
reorders                   0_0_0_0_0_0_0_0_0_0_0 1_0_1_1_0_0_1_0_0 1_1_0_...
eval_set                                                               train
Name: 0, dtype: object

In [23]:
user_id = row['user_id']
eval_set = row['eval_set']
products = row['product_ids']

In [33]:
products.split()

['15349_21413_48775_28289_8518_11777_31717_26604_43693_22475_24535',
 '43693_6808_24535_11777_21616_18761_26604_13870_13988',
 '26604_11777_20754_24231_40706',
 '27344_24535_43693_40706_16168_21413_13988_3376_8518_26604_11777_5999',
 '15349_19057_16185_21413_20843_20114_48204_40706_21616']

In [34]:
products.split()[:-1]

['15349_21413_48775_28289_8518_11777_31717_26604_43693_22475_24535',
 '43693_6808_24535_11777_21616_18761_26604_13870_13988',
 '26604_11777_20754_24231_40706',
 '27344_24535_43693_40706_16168_21413_13988_3376_8518_26604_11777_5999']

In [35]:
products, next_products = ' '.join(products.split()[:-1]), products.split()[-1]

In [36]:
reorders = row['reorders']
reorders, next_reorders = ' '.join(reorders.split()[:-1]), reorders.split()[-1]

In [37]:
product_set = set([int(j) for i in products.split() for j in i.split('_')])

In [38]:
product_set

{3376,
 5999,
 6808,
 8518,
 11777,
 13870,
 13988,
 15349,
 16168,
 18761,
 20754,
 21413,
 21616,
 22475,
 24231,
 24535,
 26604,
 27344,
 28289,
 31717,
 40706,
 43693,
 48775}

In [39]:
next_product_set = set([int(i) for i in next_products.split('_')])

In [40]:
next_product_set

{15349, 16185, 19057, 20114, 20843, 21413, 21616, 40706, 48204}

In [41]:
orders = [map(int, i.split('_')) for i in products.split()]
reorders = [map(int, i.split('_')) for i in reorders.split()]
next_reorders = map(int, next_reorders.split('_'))

In [49]:
# Instead of 'for product_id in product_set:'
product_id = next(iter(product_set))

In [50]:
product_id

28289

In [52]:
user_ids.append(user_id)

In [54]:
user_ids

[5]

In [53]:
product_ids.append(product_id)

In [55]:
product_ids

[28289]

In [56]:
labels.append(int(product_id in next_product_set) if eval_set == 'train' else -1)
labels

[0]

In [59]:
aisle_ids.append(product_to_aisle[product_id])
aisle_ids

[123, 123]

In [60]:
department_ids.append(product_to_department[product_id])
department_ids

[4, 4]

In [61]:
product_names.append(product_to_name[product_id])
product_names

['Organic Shredded Carrots', 'Organic Shredded Carrots']

In [62]:
eval_sets.append(eval_set)
eval_sets

['train', 'train']

In [63]:
is_ordered = []
index_in_order = []
order_size = []
reorder_size = []

In [64]:
prior_products = set()

In [67]:
# Instead of 'for order in orders:'
order = orders[0]